In [1]:
# Import Taichi
import taichi as ti
import taichi.math as tm
# Import numpy
import numpy as np

[Taichi] version 1.7.0, llvm 15.0.1, commit 2fd24490, win, python 3.10.6


In [2]:
ti.init(arch=ti.vulkan)

# type aliases
vec2 = tm.vec2
vec3 = tm.vec3

[Taichi] Starting on arch=vulkan


In [3]:
# define resolution
WIDTH = 800
HEIGHT = 800

PARTICLE_COUNT = 50 # define shape
PARTICLE_RADIUS = 0.02 # for rendering only

# define variables 
masses = ti.field(dtype=float, shape=PARTICLE_COUNT)
positions = ti.Vector.field(2, dtype=float, shape=PARTICLE_COUNT)
previous_positions = ti.Vector.field(2, dtype=float, shape=PARTICLE_COUNT)

@ti.func
def generate_random_float(minInclude, maxInclude):
    random_float = minInclude + (maxInclude - minInclude) * ti.random()
    return random_float

@ti.func
def random_position() -> vec2:
    return vec2(generate_random_float(0, 1), generate_random_float(0, 1))

@ti.func
def random_velocity(minInclude: vec2, maxInclude: vec2) -> vec2:
    x = generate_random_float(minInclude.x, maxInclude.x)
    y = generate_random_float(minInclude.y, maxInclude.y)
    return vec2(x, y)

@ti.kernel
def init():
    for i in range(PARTICLE_COUNT):
        positions[i] = random_position()
        previous_positions[i] = positions[i] - random_velocity(vec2(-5), vec2(5))

In [4]:
@ti.kernel
def update():
    for i in range(PARTICLE_COUNT):
        current_position = positions[i]
        # verlet simulation
        next_position = 2.0 * current_position - previous_positions[i]
        current_accleration = vec2(0)

        positions[i] = next_position + current_accleration
        previous_positions[i] = current_position
        
         # boundary constraints for x-axis
        if positions[i].x > 1:
            positions[i].x = 1
            previous_positions[i].x = next_position.x
        elif positions[i].x < 0:
            positions[i].x = 0
            previous_positions[i].x = next_position.x 

        # boundary constraints for y-axis
        if positions[i].y > 1:
            positions[i].y = 1
            previous_positions[i].y = next_position.y
        elif positions[i].y < 0:
            positions[i].y = 0
            previous_positions[i].y = next_position.y

In [5]:
init()

window = ti.ui.Window("Mass Spring System", (WIDTH, HEIGHT), vsync=True)
canvas = window.get_canvas()
canvas.set_background_color((1, 1, 1))

scene = window.get_scene()
camera = ti.ui.Camera()
camera.position(0, 0, 5)
camera.lookat(0, 0, 0) 
camera.up(0, 1, 0) 

while window.running:
    # update()
    
    # Setting up the camera
    camera = ti.ui.make_camera()
    camera.position(0, 0, 5)
    camera.lookat(0, 0, 0)
    camera.up(0, 1, 0)
    scene.set_camera(camera)

    scene.point_light(pos=(0, 10, 10), color=(1, 1, 1))
    scene.ambient_light(color=(0.5, 0.5, 0.5))

    collide_sphere = ti.Vector.field(3, dtype=float, shape=(1, ))
    collide_sphere[0] = [0, 0, 0]
    scene.particles(collide_sphere, radius=1, color=(0.5, 0.5, 0.5))
    canvas.scene(scene)
    window.show()